In [1]:
from IPython.display import display, HTML
from ast import literal_eval
# import spacy
import nltk
from tqdm import tqdm
import pandas as pd

In [2]:
tqdm.pandas()

In [3]:
sample_tweets_file = '../tweets/actors/tweets.csv'

In [4]:
df = pd.read_csv(sample_tweets_file, low_memory=False)

In [5]:
# date
# time
# author
# group 

# TODO: do it on clean tweets:
# tweet words
# tweet len

# TODO: overall post sentiment ('positive'/'negative')

# mentions num
# hashtags num

# has an url
# has an image

# has a gif
# has a movie
# has a place

# replies_count, retweets_count, likes_count

# is a reply
# is a quote

In [6]:
group = 'actors'

In [7]:
df['tp_date'] = df.progress_apply(lambda tweet: tweet.date, axis=1)
df['tp_time'] = df.progress_apply(lambda tweet: tweet.time, axis=1)
df['tp_author'] = df.progress_apply(lambda tweet: tweet.username, axis=1)
df['tp_group'] = group

# TODO: use clean tweet here (no emojis, stop words, etc.)
df['tp_words_num'] = df.progress_apply(lambda tweet: len(nltk.word_tokenize(tweet.tweet)), axis=1)
df['tp_tweet_len'] = df.progress_apply(lambda tweet: len(tweet.tweet), axis=1)

df['tp_sentiment'] = '?'

df['tp_mentions_num'] = df.progress_apply(lambda tweet: len(literal_eval(tweet.mentions)), axis=1)
df['tp_hashtags_num'] = df.progress_apply(lambda tweet: len(literal_eval(tweet.tags)), axis=1)

df['tp_has_url'] = df.progress_apply(lambda tweet: bool(len(literal_eval(tweet.urls))), axis=1)
df['tp_has_image'] = df.progress_apply(lambda tweet: bool(len(literal_eval(tweet.photos))), axis=1)

df['tp_has_gif'] = df.progress_apply(lambda tweet: not pd.isnull(tweet.gif_url), axis=1)
df['tp_has_video'] = df.progress_apply(lambda tweet: not pd.isnull(tweet.video_url), axis=1)
df['tp_has_place'] = df.progress_apply(lambda tweet: not pd.isnull(tweet.place), axis=1)

df['tp_replies_count'] = df.progress_apply(lambda tweet: tweet.replies_count, axis=1)
df['tp_retweets_count'] = df.progress_apply(lambda tweet: tweet.retweets_count, axis=1)
df['tp_likes_count'] = df.progress_apply(lambda tweet: tweet.likes_count, axis=1)

df['tp_is_reply'] = df.progress_apply(lambda tweet: bool(tweet.is_reply_to), axis=1)
df['tp_is_quote'] = df.progress_apply(lambda tweet: bool(tweet.is_quote_status), axis=1)

100%|██████████| 612672/612672 [00:16<00:00, 37763.43it/s]


In [8]:
cols_to_rm = [c for c in df.columns if not c.startswith('tp_')]

In [9]:
final_df = df.drop(cols_to_rm, axis=1)

In [10]:
with pd.option_context('display.max_rows', 500, 'display.max_columns', 100, 'display.max_colwidth', -1):
    display(final_df)

,tp_date,tp_time,tp_author,tp_group,tp_words_num,tp_tweet_len,tp_sentiment,tp_mentions_num,tp_hashtags_num,tp_has_url,tp_has_image,tp_has_gif,tp_has_video,tp_has_place,tp_replies_count,tp_retweets_count,tp_likes_count,tp_is_reply,tp_is_quote
0,2018-11-14,19:52:39,evalongoria,actors,44,299,?,1,0,True,True,False,False,False,11,49,223,False,False
1,2018-11-13,20:08:20,evalongoria,actors,31,152,?,2,0,False,True,False,False,False,28,79,1006,False,False
2,2018-11-10,19:58:27,evalongoria,actors,7,60,?,0,0,False,True,False,False,False,150,273,6693,False,False
3,2018-11-09,23:40:07,evalongoria,actors,45,257,?,3,0,False,True,False,False,False,22,73,732,False,False
4,2018-04-13,20:27:38,justdemi,actors,29,181,?,1,0,True,False,False,False,False,207,67,295,False,True
5,2018-11-09,21:07:44,evalongoria,actors,42,298,?,0,0,True,False,False,False,False,5,56,190,False,True
6,2018-03-29,07:12:44,justdemi,actors,37,224,?,2,0,True,False,False,False,False,77,94,701,False,True
7,2018-11-08,17:53:04,evalongoria,actors,30,164,?,1,0,False,True,False,False,False,8,55,461,False,False
8,2018-11-14,13:07:22,michaelianblack,actors,10,36,?,0,0,False,False,False,False,False,2,0,45,True,False
9,2018-11-06,20:28:41,evalongoria,actors,33,160,?,0,0,False,True,False,False,False,33,164,1558,False,False
